In [1]:
import glob
import re
import tensorflow as tf
import random
import numpy as np

In [2]:
def load_filenames_labels(mode):
    """Gets filenames and labels
    Args:
    mode: 'train' or 'val'
      (Directory structure and file naming different for
      train and val datasets)
    Returns:
    list of tuples: (jpeg filename with path, label)
    """
    label_dict, class_description = build_label_dicts()
    filenames_labels = []
    if mode == 'train':
        filenames = glob.glob('tiny-imagenet-200/*/images/*.JPEG')
        for filename in filenames:
            match = re.search(r'n\d+', filename)
            if match is None:
                return None
            label = str(label_dict[match.group()])
            filenames_labels.append((filename, label))
    elif mode == 'val':
        with open('tiny-imagenet-200/val/val_annotations.txt', 'r') as f:
            for line in f.readlines():
                split_line = line.split('\t')
                filename = 'tiny-imagenet-200/val/images/' + split_line[0]
                label = str(label_dict[split_line[1]])
                filenames_labels.append((filename, label))
    return filenames_labels

In [3]:
def build_label_dicts():
    """Build look-up dictionaries for class label, and class description
    Class labels are 0 to 199 in the same order as 
    tiny-imagenet-200/wnids.txt. Class text descriptions are from 
    tiny-imagenet-200/words.txt
    
    Returns:
    tuple of dicts
    label_dict: 
    keys = synset (e.g. "n01944390")
    values = class integer {0 .. 199}
    class_desc:
    keys = class integer {0 .. 199}
    values = text description from words.txt
    """
    label_dict, class_description = {}, {}
    with open('tiny-imagenet-200/wnids.txt', 'r') as f:
        for i, line in enumerate(f.readlines()):
            synset = line[:-1]  # remove \n
            label_dict[synset] = i
    with open('tiny-imagenet-200/words.txt', 'r') as f:
        for i, line in enumerate(f.readlines()):
            synset, desc = line.split('\t')
            desc = desc[:-1]  # remove \n
            if synset in label_dict:
                class_description[label_dict[synset]] = desc
    return label_dict, class_description

In [4]:
def read_image(filename_q, mode):
    """Load next jpeg file from filename / label queue
    Randomly applies distortions if mode == 'train' (including a 
    random crop to [56, 56, 3]). Standardizes all images.

    Args:
    filename_q: Queue with 2 columns: filename string and label string.
    filename string is relative path to jpeg file. label string is text-
    formatted integer between '0' and '199'
    mode: 'train' or 'val'

    Returns:
    [img, label]: 
    img = tf.uint8 tensor [height, width, channels]  (see tf.image.decode.jpeg())
    label = tf.unit8 target class label: {0 .. 199}
    """
    item = filename_q.dequeue()
    filename = item[0]
    label = item[1]
    file = tf.read_file(filename)
    img = tf.image.decode_jpeg(file, channels=3)
    # image distortions: left/right, random hue, random color saturation
    if mode == 'train':
        img = tf.random_crop(img, np.array([56, 56, 3]))
        img = tf.image.random_flip_left_right(img)
        # val accuracy improved without random hue
        # img = tf.image.random_hue(img, 0.05)
        img = tf.image.random_saturation(img, 0.5, 2.0)
    else:
        img = tf.image.crop_to_bounding_box(img, 4, 4, 56, 56)

    label = tf.string_to_number(label, tf.int32)
    label = tf.cast(label, tf.uint8)

    return [img, label]

In [5]:
def batch_q(mode, config):
    """Return batch of images using filename Queue

    Args:
    mode: 'train' or 'val'
    config: training configuration object

    Returns:
    imgs: tf.uint8 tensor [batch_size, height, width, channels]
    labels: tf.uint8 tensor [batch_size,]
    """
    filenames_labels = load_filenames_labels(mode)
    random.shuffle(filenames_labels)
    filename_q = tf.train.input_producer(filenames_labels,num_epochs=config.num_epochs,shuffle=True)

    # 2 read_image threads to keep batch_join queue full:
    return tf.train.batch_join([read_image(filename_q, mode) for i in range(2)],config.batch_size, shapes=[(56, 56, 3), ()],capacity=2048)

In [6]:
load_filenames_labels('train')

In [7]:
build_label_dicts()

({'n01443537': 22,
  'n01629819': 55,
  'n01641577': 5,
  'n01644900': 183,
  'n01698640': 160,
  'n01742172': 177,
  'n01768244': 174,
  'n01770393': 125,
  'n01774384': 141,
  'n01774750': 43,
  'n01784675': 161,
  'n01855672': 67,
  'n01882714': 68,
  'n01910747': 94,
  'n01917289': 164,
  'n01944390': 33,
  'n01945685': 100,
  'n01950731': 165,
  'n01983481': 151,
  'n01984695': 89,
  'n02002724': 115,
  'n02056570': 41,
  'n02058221': 35,
  'n02074367': 116,
  'n02085620': 182,
  'n02094433': 135,
  'n02099601': 78,
  'n02099712': 39,
  'n02106662': 11,
  'n02113799': 194,
  'n02123045': 66,
  'n02123394': 131,
  'n02124075': 0,
  'n02125311': 102,
  'n02129165': 184,
  'n02132136': 14,
  'n02165456': 76,
  'n02190166': 113,
  'n02206856': 87,
  'n02226429': 180,
  'n02231487': 181,
  'n02233338': 48,
  'n02236044': 51,
  'n02268443': 105,
  'n02279972': 13,
  'n02281406': 42,
  'n02321529': 37,
  'n02364673': 17,
  'n02395406': 101,
  'n02403003': 120,
  'n02410509': 175,
  'n024